In [ ]:
# ✅ STEP 1: Install Required Libraries
!pip install nltk gradio joblib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.9 MB/s eta 0:00:00


In [ ]:
# ✅ STEP 2: Import Libraries
import pandas as pd
import numpy as np
import nltk
import re
import string
import gradio as gr
import joblib

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# ✅ STEP 3: Upload and Load Dataset
from google.colab import files
uploaded = files.upload()

# Instead of 'error_bad_lines' and 'warn_bad_lines',
# Use 'on_bad_lines' to handle problematic lines in newer pandas versions.
# This will skip lines that cause parsing errors, potentially losing some data but allowing the code to continue.
fake_df = pd.read_csv('Fake.csv', on_bad_lines='skip', encoding='utf-8') # Changed from error_bad_lines to on_bad_lines
real_df = pd.read_csv('True.csv', on_bad_lines='skip', encoding='utf-8') # Changed from error_bad_lines to on_bad_lines

Saving Fake.csv to Fake.csv
Saving True.csv to True.csv


In [ ]:
# Add labels
fake_df['label'] = 0  # Fake
real_df['label'] = 1  # Real

# Combine and shuffle
data = pd.concat([fake_df, real_df], ignore_index=True)
data = data.sample(frac=1).reset_index(drop=True)

# ✅ STEP 4: Preprocess Text
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def preprocess(text):
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>+', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    words = text.split()
    words = [stemmer.stem(word) for word in words if word not in stop_words]
    return ' '.join(words)


In [ ]:
# Apply preprocessing
data['content'] = data['title'] + " " + data['text']
data['content'] = data['content'].apply(preprocess)

# ✅ STEP 5: Split Data
X = data['content']
y = data['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# ✅ STEP 6: Vectorize
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# ✅ STEP 7: Train Classifier
model = MultinomialNB()
model.fit(X_train_vect, y_train)

# ✅ STEP 8: Evaluate
y_pred = model.predict(X_test_vect)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# ✅ STEP 9: Save Model and Vectorizer
joblib.dump(model, "fake_news_model.pkl")
joblib.dump(vectorizer, "vectorizer.pkl")





Accuracy: 0.9327394209354121
              precision    recall  f1-score   support

           0       0.93      0.94      0.94      4737
           1       0.93      0.93      0.93      4243

    accuracy                           0.93      8980
   macro avg       0.93      0.93      0.93      8980
weighted avg       0.93      0.93      0.93      8980



['vectorizer.pkl']

In [ ]:
# ✅ STEP 10: Gradio Web App
import gradio as gr
import joblib  # Import joblib for loading the model and vectorizer

# Load the saved model and vectorizer
model = joblib.load("fake_news_model.pkl")
vectorizer = joblib.load("vectorizer.pkl")

# Define the predict_news function
def predict_news(news_text):
    # Preprocess the input text
    processed_text = preprocess(news_text)
    # Vectorize the processed text
    vectorized_text = vectorizer.transform([processed_text])
    # Make the prediction

In [ ]:
import joblib  # Use joblib instead of pickle

# Load trained model and vectorizer
with open("fake_news_model.pkl", "rb") as f:
    model = joblib.load(f)  # Use joblib.load()

with open("vectorizer.pkl", "rb") as f:
    vectorizer = joblib.load(f)  # Use joblib.load()

# Example news
news_text = "China announces the successful launch of its new space station module, expanding its space program."

# Preprocess and vectorize
news_vector = vectorizer.transform([news_text])

# Predict
prediction = model.predict(news_vector)

# Output
if prediction[0] == 1:
    print("Real News ✅")
else:
    print("Fake News ❌")

Real News ✅


In [ ]:
def predict_news(news):
    input_vec = vectorizer.transform([news])
    prediction = model.predict(input_vec)[0]
    return "🟢 Real News" if prediction == 1 else "🔴 Fake News"

# ✅ Gradio Web Interface
with gr.Blocks() as interface:
    gr.Markdown("<h1 style='text-align: center;'>📰 Fake News Detector By naive_bayes</h1>")
    gr.Markdown("<p style='text-align: center;'>Enter a news article and let AI tell you if it's Real or Fake!</p>")

    with gr.Row():
        news_input = gr.Textbox(label="✏️ Enter News Article", placeholder="Paste news article here...", lines=10)

    with gr.Row():
        submit_btn = gr.Button("🔍 Detect")
        result_output = gr.Textbox(label="📢 Prediction", interactive=False)

    submit_btn.click(fn=predict_news, inputs=news_input, outputs=result_output)

interface.launch(share=True, debug=False)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e44a8519acfa804f05.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:

# ✅ STEP 11: Download Buttons in Colab
from google.colab import files
files.download('fake_news_model.pkl')
files.download('vectorizer.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login
notebook_login()


In [ ]:
from huggingface_hub import HfApi, HfFolder, Repository
from pathlib import Path
import shutil

# Define repo name
repo_name = "fake-news-detector-model"  # or your custom name
model_path = Path(repo_name)
model_path.mkdir(exist_ok=True)

# ✅ Copy your model and vectorizer to the folder
shutil.copy("fake_news_model.pkl", model_path / "fake_news_model.pkl")
shutil.copy("vectorizer.pkl", model_path / "vectorizer.pkl")

PosixPath('fake-news-detector-model/vectorizer.pkl')

In [ ]:
from huggingface_hub import create_repo, upload_folder

# Replace 'your-username' with your actual Hugging Face username
create_repo(repo_name, exist_ok=True)

upload_folder(
    folder_path=model_path,
    repo_id="AAFREEN-KAZMI/fake-news-detector-model",  # ✅ Replace with your actual username
    path_in_repo=".",
    repo_type="model"
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

fake_news_model.pkl:   0%|          | 0.00/161k [00:00<?, ?B/s]

vectorizer.pkl:   0%|          | 0.00/180k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/AAFREEN-KAZMI/fake-news-detector-model/commit/2147f228315cf3cfbb5ef704fcd07da780e0e759', commit_message='Upload folder using huggingface_hub', commit_description='', oid='2147f228315cf3cfbb5ef704fcd07da780e0e759', pr_url=None, repo_url=RepoUrl('https://huggingface.co/AAFREEN-KAZMI/fake-news-detector-model', endpoint='https://huggingface.co', repo_type='model', repo_id='AAFREEN-KAZMI/fake-news-detector-model'), pr_revision=None, pr_num=None)

In [1]:
!pip install --upgrade jupyter


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 115.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.8/385.8 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.2 MB/s eta 0:00:00
  Attempting uninstall: jupyter-client
    Found existing installation: jupyter-client 6.1.12
    Uninstalling jupyter-client-6.1.12:
      Successfully uninstalled jupyter-client-6.1.12
  Attempting uninstall: jupyter-server
    Found existing installation: jupyter-server 1.16.0
    Uninstalling jupyter-server-1.16.0:
      Successfully uninstalled jupyter-server-1.16.0


In [2]:
!pip install --upgrade nbconvert
